In [1]:
from deepparse.parser import AddressParser

/Users/adrienblanquer/code/bas/bas-utils/.venv/lib/python3.12/site-packages/pymagnitudelight/framework/repoze/__init__.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


In [3]:
address_parser = AddressParser(model_type="fasttext", device=0)

/Users/adrienblanquer/code/bas/bas-utils/.venv/lib/python3.12/site-packages/deepparse/parser/address_parser.py:1092: UserWarning: No CUDA device detected, device will be set to 'CPU'.
  warnings.warn("No CUDA device detected, device will be set to 'CPU'.", category=UserWarning)
/Users/adrienblanquer/code/bas/bas-utils/.venv/lib/python3.12/site-packages/deepparse/download_tools.py:92: UserWarning: The offline parameter is set to False, so if a new pre-trained `fasttext` model is available it will automatically be downloaded.
  warnings.warn(


The FastText pretrained word embeddings will be downloaded (6.8 GO), this process will take several minutes.
(100.00%) [==================================================>]
Loading the embeddings model


In [9]:
import duckdb
import pandas as pd
import os
from dotenv import load_dotenv


In [10]:
duck = duckdb.connect()

In [11]:
duck.execute("load postgres")
load_dotenv("../.env")

duck.execute(
    f"""
    create or replace secret (
        type postgres,
        host '{os.environ.get('DB_HOST', 'localhost')}',
        port {os.environ.get('DB_PORT', 5432)},
        database '{os.environ.get('DB_NAME', 'postgres')}',
        user '{os.environ.get('DB_USER', 'postgres')}',
        password '{os.environ.get('DB_PASSWORD', 'postgres')}'
        );
    
    attach '' as pg (type postgres);
    """
)

In [13]:
duck.execute(
    """
    begin transaction;

    create or replace temp table zoho_accounts as
    select * replace(trim(Account_Name) as Account_Name) from pg.zoho.Accounts where Account_Status != 'In Akquisition';

    create or replace temp table medisoft_firms as
    select * replace (trim(name) as name, trim(kuerzel) as kuerzel) from pg.medisoft.table_firmenstruktur;

    commit;
    """
)

In [14]:
med_df = duck.sql(
    """
    select rec_id, name, plz, strasse from medisoft_firms where rec_id not in (select rec_id from pg.medisoft.table_firms_zoho where id_zoho is not null)
    """
).df()

In [16]:
med_clean_df = med_df.dropna(ignore_index=True)
med_clean_df['raw_address'] = med_clean_df['strasse'] + ", " + med_clean_df['plz'] + ", Deutschland"

/var/folders/99/6ltr_n_s7fq3lxwcq9fhbpdw0000gq/T/ipykernel_48089/1323936621.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  med_clean_df['raw_address'] = med_clean_df['strasse'] + ", " + med_clean_df['plz'] + ", Deutschland"


In [17]:
address_parser(med_clean_df['raw_address'])

Vectorizing the address


[FormattedParsedAddress<StreetNumber='32', StreetName='karl-liebknecht str.', Municipality='deutschland', PostalCode='10178'>,
 FormattedParsedAddress<StreetNumber='185', StreetName='rudolf-breidtscheidstrasse', Municipality='deutschland', PostalCode='14482'>,
 FormattedParsedAddress<StreetNumber='9', StreetName='krokusstr.', Municipality='deutschland', PostalCode='12357'>,
 FormattedParsedAddress<StreetNumber='248', StreetName='karl-marx-str.', Municipality='deutschland', PostalCode='12057'>,
 FormattedParsedAddress<StreetNumber='2', StreetName='rosa-luxemburg-straße', Municipality='deutschland', PostalCode='10178'>,
 FormattedParsedAddress<StreetNumber='255', StreetName='karl-marx-str.', Municipality='deutschland', PostalCode='12057'>,
 FormattedParsedAddress<StreetNumber='111', StreetName='frankfurter allee', Municipality='deutschland', PostalCode='10247'>,
 FormattedParsedAddress<StreetNumber='16', StreetName='schudomarstr.', Municipality='deutschland', PostalCode='12055'>,
 Format